In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
classification_report, accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support
)
import pickle
import time

In [2]:
df = pd.read_csv('../data/cleaned/311_Data_Cleaned_Modeling.csv.bz2', compression = 'bz2')
df.head()

,agency_acronym,borough,complaint_type,location_type,time_of_day,incident_zip
0,NYPD,MANHATTAN,Noise Complaints,Commercial,Evening,10031
1,NYPD,MANHATTAN,Noise Complaints,Residential,Evening,10013
2,NYPD,MANHATTAN,Noise Complaints,Commercial,Evening,10031
3,DPR,STATEN ISLAND,Animal-Related Complaints,Outdoor,Evening,10314
4,NYPD,MANHATTAN,Noise Complaints,Residential,Evening,10013


### Preprocess Data

The first step is to split the data into X and y features and then to perform a train-test split. Although it should not matter for our dataset, generally it is best to perform a train-test split before doing encoding and scaling so that there is no data leakage. For example, if using StandardScaler on the full dataset, the StandardScaler will learn mean and standard deviation from testing data which is poor practice.

In [3]:
# Define features and target
X = df.drop('agency_acronym', axis = 1)
y = df['agency_acronym']

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

We need to convert all of the columns in X into one-hot encoded columns using OneHotEncoder, as they are all categorical. If there were any numerical columns, it would be best to scale them as well using StandardScaler, but that is not necessary in this case. We do not drop any of the columns when one-hot-encoding, because even though this introduces multicollinearity, we will handle this multicollinearity via regularization.

In [4]:
# Because all of the columns in X are categorical, we use all of the columns for OneHotEncoding
categorical_columns = X.columns.tolist()

# Create the ColumnTransformer with OneHotEncoder
# Not dropping first as multicollinearity will be handled by regularization
ohe = ColumnTransformer(
    transformers = [
        ('onehot', OneHotEncoder(), categorical_columns)
    ],
    remainder = 'drop'
)

# Fit the preprocessor to the training data and transform the training and testing data
# Sparse matrices to speed up calculations and save memory
X_train = ohe.fit_transform(X_train)
X_test = ohe.transform(X_test)

Next, we need to convert the target variable into numerical outputs using LabelEncoder so that the values are from 0 to n_classes - 1 rather than as strings.

In [5]:
# Instantiate encoder
label_encoder = LabelEncoder()

# Fit and transform y series
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

### Training

#### Helper Functions

Before training, it is important to consider what an appropriate scoring metric is for using the GridSearch scoring.

In the context of this project's task, Precision measures the ability of the model to correctly direct the caller to the correct agency. A low Precision means that callers are often erroneously directed to that agency. While it is possible that the caller could be redirected to the appropriate agency afterwards, that scenario involves wasted time and manpower spent redirecting the call which is not ideal when dealing with agencies such as the NYPD. Thus, Precision is a valuable metric.

Recall measures the ability of the model to correctly capture all of the callers that should be directed to the correct agency. A low Recall means that the callers that should be directed to a given agency are not being directed there. This leads to lower resolution rates, longer times to resolve problems, and a general lack of effectiveness in the 311 system. Therefore, Recall is a valuable metric.

Since both Precision and Recall are important, the score we will maximize is the Macro F1-Score. Macro F1-Score is used over Weighted because it treats each class as equally important, which is necessary because there are large class imbalances and it would not be helpful for the more prevalent classes to dominate the less prevalent ones. Because there are class imbalances, Accuracy is not the most reliable metric and will not be used as the primary scorer.

We will use 3-fold cross-validation in the GridSearchCV to perform hyperparameter tuning. The number of folds needs to be low because the size of the data is large enough that training takes a long time. The entire training process will be timed to get a sense of computational complexity and along with the fit time for the best found model for each grid search they will be put into a dictionary. The best model will also be captured and added to the dictionary before finally being pickled to allow for loading at a later time.

In [6]:
# Helper function to train models
def run_gridsearch_and_save(classifier, param_grid, X_train, y_train, pickle_filename):
    # Record start time for the overall fitting process
    start_time = time.time()

    # Setup GridSearchCV with f1_macro as the scoring metric
    grid_search = GridSearchCV(
        estimator = classifier,
        param_grid = param_grid,
        cv = 3, # Reduced number of cross-validation folds to save time due to dataset size
        scoring = 'f1_macro',
        n_jobs = -1,
        verbose = 1
    )
    
    # Fit the model
    grid_search.fit(X_train, y_train)

    # Record total fit time
    total_fit_time = time.time() - start_time

    # Extract relevant information
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_model_fit_time = grid_search.cv_results_['mean_fit_time'][grid_search.best_index_]
    num_models_tested = len(grid_search.cv_results_['params'])

    # Create a dictionary to store the results
    result_dict = {
        'model_name': classifier.__class__.__name__,
        'best_model': best_model,
        'total_fit_time': total_fit_time,
        'num_models_tested': num_models_tested,
        'best_model_fit_time': best_model_fit_time
    }

    # Save the result dictionary to a pickle file
    with open(pickle_filename, 'wb') as f:
        pickle.dump(result_dict, f)

    print(f"Grid search completed. Results saved to {pickle_filename}")

    return

In [7]:
# Helper function to load pickled models
def load_pickle(path):
    with open(path, 'rb') as f:
        model = pickle.load(f)

    return model

#### Model Fitting / Loading

The four model types tested are Logistic Regression via the SGDClassifier (in order to implement Stochastic Gradient Descent), a DecisionTreeClassifier and RandomForestClassifier (non-parametric models that cannot use Gradient Descent), and finally an XGBClassifier to test a boosted model that has its own learning rate. The parameter grids are deliberately small to cut down on training time. It is not ideal, but the size of the data and power of the machines available to us restricts our ability to do a more exhaustive grid search. Details of the parameters searched are noted in each cell. Each of the models were trained in Google Colab using a high memory instance (hence no print statements in this notebook).

Logistic regression is tested with various levels of regularization strength and testing different balances of L1 (Lasso) vs. L2 (Ridge) regularization using ElasticNet. To take advantage of the SGDClassifier, different learning rates are testing using the adaptive strategy which keeps the learning rate the same until no changes happen then divides the learning rate by 5 and continues.

The best model used a starting learning rate of 0.1, an L1 ratio of 0.85 and the default regularization strength of 0.0001. This parameter grid fit 27 different models using 3-fold validation for a total of 81 models and had the second worst model fit time of \~387 seconds (\~6.5 minutes).

In [8]:
# LogisticRegression via SGDClassifier
param_grid = {
    'loss': ['log_loss'],               # Log loss makes SGDClassifier equivalent to LogisticRegression
    'penalty': ['elasticnet'],          # Use the ElasticNet regularization penalty
    'alpha': [1e-4, 1e-3, 1e-2],        # Regularization strength
    'l1_ratio': [0.15, 0.5, 0.85],      # Percentage of the regularization weighted towards L1 vs. L2
    'learning_rate': ['adaptive'],      # Setting the learning rate to adaptive to adjust learning rate
    'eta0': [0.001, 0.01, 0.1],         # Initial learning rate
    'max_iter': [1000],                 # Maximum iterations for each model
    'tol': [1e-3],                      # Stopping criterion
}

# run_gridsearch_and_save(SGDClassifier(), param_grid, X_train, y_train, '../models/logistic_regression.pkl')

logistic_regression = load_pickle('../models/logistic_regression.pkl')
logistic_regression

{'model_name': 'SGDClassifier',
 'best_model': SGDClassifier(eta0=0.1, l1_ratio=0.85, learning_rate='adaptive',
               loss='log_loss', penalty='elasticnet'),
 'total_fit_time': 3945.7306056022644,
 'num_models_tested': 27,
 'best_model_fit_time': 387.56030400594074}

The Decision Tree is tested with a few different options for maximum depth and minimum samples split. Decision Trees are prone to overfitting so it is often helpful to reduce the maximum depth or increase the minimum samples required to create splits. Increasing the minimum samples per leaf helps prevent the tree from making sparse branches that separate very small amounts of data. Weighting based on class balances was tested to see if it was helpful due to class imbalances.

The best model had the default values for max depth (None) and minimum samples split (2), likely due to the extensive preprocessing done on the data which helped to prevent extreme overfitting. The minimum samples leaf being slightly increased to 4 (from the default of 1) was beneficial. Weighting for class balances was not beneficial. 24 different models were fit using 3-fold validation for a total of 72 models. The parameter grid could possibly have been made larger, but it was kept fairly small for time constraints. The decision tree had the second fastest fit time of 195 seconds (3.25 minutes).

In [9]:
# DecisionTreeClassifier
param_grid = {
    'max_depth': [10, 20, None],        # Maximum depth of tree to reduce complexity and avoid overfitting
    'min_samples_split': [2, 10],       # Minimum samples to split to reduce complexity and avoid overfitting
    'min_samples_leaf': [1, 4],         # Minimum samples per leaf to reduce complexity
    'class_weight': [None, 'balanced']  # Weights for each class to address class imbalances
}
# run_gridsearch_and_save(DecisionTreeClassifier(random_state = 42), param_grid,
                        # X_train, y_train, '../models/decision_tree.pkl')

decision_tree = load_pickle('../models/decision_tree.pkl')
decision_tree

{'model_name': 'DecisionTreeClassifier',
 'best_model': DecisionTreeClassifier(min_samples_leaf=4, random_state=42),
 'total_fit_time': 1043.8672311306,
 'num_models_tested': 24,
 'best_model_fit_time': 195.22301697731018}

The Random Forest was tested with similar parameters to the Decision Tree. The max depth for each tree does not need to be the as large as the Decision Trees, as the Random Forest is meant to use an ensemble of weaker learners. Random Forest is notably much slower to train, so the parameter grid is deliberately very small to help reduce training time. Random Forests tend to perform well without significant hyperparameter tuning.

The best model used a max depth of 10 with 200 estimators and the default minimum samples split of 2 (identical to the performance of the Decision Tree). The model likely could have benefitted from a larger number of estimators due to the sheer size of the dataset but training times made that impractical. 8 models were fit using 3-fold cross validation for a total of 24 models. The Random Forest had the worst computational complexity performance with a fit time of 673 seconds (~11 minutes).

In [10]:
# RandomForest Classifier
param_grid = {
    'n_estimators': [100, 200],          # Number of trees to use for boosting
    'max_depth': [5, 10],                # Maximum depth of each tree to reduce complexity and avoid overfitting
    'min_samples_split': [2, 10],        # Minimum samples to split to reduce complexity and avoid overfitting
}
# run_gridsearch_and_save(RandomForestClassifier(random_state = 42), param_grid,
#                         X_train, y_train, '../models/random_forest.pkl')

random_forest = load_pickle('../models/random_forest.pkl')
random_forest

{'model_name': 'RandomForestClassifier',
 'best_model': RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42),
 'total_fit_time': 1856.613802909851,
 'num_models_tested': 8,
 'best_model_fit_time': 673.4580012162527}

Note that XGBoost has a different setup than the typical sklearn models. The function for fitting and training the models was not customized to adjust for the XGBoost architecture, which means that the pickled models must be trained and loaded on the same version of XGBoost. The version of XGBoost used was version 2.1.4.

In [11]:
# !pip install xgboost==2.1.4

XGBoost is a boosted model, so it benefits from various learning rates and a higher number of boosting rounds. The max depth can be much lower than normal decision trees or random forest since each tree learns from the prior trees. Subsample helps to control the fraction of samples used for each tree which can help prevent overfitting by not letting each tree fit on the entire training data. XGBoost also utilizes L1 and L2 regularizations.

The best model utilized a max depth of 6 with 0.01 L1 regularization and 1 L2 regularization. This has minimal regularization, but the risk of overfitting is reduced by the low max depth and by using a subsample fraction of 0.8. The best model used the larger learning rate of 0.1. It would likely be beneficial to use more estimators to have more boosting rounds, although it is possible to have too many boosting rounds and overfit the data.

Notably, XGBoost is able to utilize GPU resources for faster training. The XGBoost model was fit on the A100 GPU on Google Colab. 64 models were fit with 3-fold cross validation for a total of 192 models. The XGBoost fit 192 models in less time than the Random Forest fit 24 models. XGBoost had the best computational performance with a fit time of only 167 seconds  (~2.75 minutes).

In [12]:
# XGBoostClassifier
param_grid = {
    'n_estimators': [100, 200],          # Number of trees to use for boosting (number of boosting rounds)
    'learning_rate': [0.05, 0.1],        # Learning rate: step size for each boosting round
    'max_depth': [3, 6],                 # Maximum depth of trees
    'subsample': [0.8, 1.0],             # Fraction of samples used for each tree
    'reg_alpha': [0.01, 0.1],            # L1 regularization term on weights
    'reg_lambda': [1, 1.5]               # L2 regularization term on weights
}
# run_gridsearch_and_save(XGBClassifier(random_state = 42), param_grid, X_train, y_train, '../models/xgboost.pkl')

xgboost = load_pickle('../models/xgboost.pkl')
# The XGBoost model was trained on a GPU
# The tree method needs to be readjusted when loading on a model without GPU support
xgboost['best_model'].tree_method = 'auto'
xgboost

{'model_name': 'XGBClassifier',
 'best_model': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.1, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=6, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=200, n_jobs=None,
               num_parallel_tree=None, objective='multi:softprob', ...),
 'total_fit_time': 1687.4453814029694,
 'num_models_tested': 64,
 'best_model_fit_time': 167.11927111943564}

In [13]:
models = [logistic_regression, decision_tree, random_forest, xgboost]

### Scoring, Evaluation, and Model Selection

#### Helper Functions

In [ ]:
# Helper function to plot learning curves
def plot_learning_curves(models, X, y, cv = 3,
                         train_sizes = [0.1, 0.3, 0.5, 0.8, 1],
                         random_state = 42, save_path = None):
    
    n_models = len(models)
    
    # Create subplots: one column per model
    fig, axes = plt.subplots(1, n_models, figsize = (6 * n_models, 5), sharey = True)
    
    # If only one model, make axes a list
    if n_models == 1:
        axes = [axes]
    
    # Define a color palette
    colors = plt.cm.tab10.colors  # 10 distinct colors
    
    # Define the macro F1 scorer
    scorer = make_scorer(f1_score, average = 'macro')
    
    for idx, (model_info, ax) in enumerate(zip(models, axes)):
        
        # Calculate learning curves
        train_sizes_abs, train_scores, val_scores = learning_curve(
            model_info["best_model"], X, y,
            cv = cv,
            train_sizes = train_sizes,
            scoring = scorer,
            random_state = random_state,
            n_jobs = -1
        )
        
        # Compute mean scores across folds
        train_scores_mean = np.mean(train_scores, axis = 1)
        val_scores_mean = np.mean(val_scores, axis = 1)
        
        # Convert training sizes to percentages
        train_sizes_pct = 100 * train_sizes_abs / X.shape[0]
        
        # Plot validation and training scores
        color = colors[idx % len(colors)]
        ax.plot(train_sizes_pct, val_scores_mean, label = "Validation", color = color, linestyle = '-')
        ax.plot(train_sizes_pct, train_scores_mean, label = "Training", color = color, linestyle = '--')
        
        # Format the axis using the helper
        ylabel = "Macro F1-score" if idx == 0 else ""  # Only first plot shows y-label
        format_axis(ax, model_info['model_name'], "Training Set Size (%)", ylabel)
        
        ax.grid(True)
        ax.legend()
    
    # Set a main title
    fig.suptitle("Learning Curves (Macro F1-score)", fontsize = 24)
    fig.tight_layout(rect = [0, 0, 1, 0.93])
    
    # Save the figure if requested
    if save_path is not None:
        with open(save_path, 'wb') as f:
            pickle.dump(fig, f)
    
    plt.show()

In [14]:
# Helper function to generate baseline model (random guessing based on class distribution)
def generate_random_baseline(y):
    # Get class probabilities
    classes = np.unique(y)
    class_counts = np.bincount(y)
    class_probabilities = class_counts / len(y)
    
    # Generate random predictions based on class distribution
    random_predictions = np.random.choice(classes, size = len(y), p = class_probabilities)

    # Calculate metrics for the random baseline
    accuracy = accuracy_score(y, random_predictions)
    macro_precision = precision_score(y, random_predictions, average = 'macro', zero_division = 0)
    macro_recall = recall_score(y, random_predictions, average = 'macro', zero_division = 0)
    macro_f1 = f1_score(y, random_predictions, average = 'macro', zero_division = 0)

    return accuracy, macro_precision, macro_recall, macro_f1

In [15]:
# Helper function for generating a condensed classification report table as a Pandas Styled object
def generate_classification_report_table(models_list, X_test, y_test):
    records = []
    
    # Get metrics for baseline model (using y_test) that predicts according to class probabilities
    baseline_metrics = generate_random_baseline(y_test)
    
    # Append baseline model to records
    records.append({
        'Model Name': 'Random Baseline',
        'Accuracy': round(baseline_metrics[0], 3),
        'Macro Precision': round(baseline_metrics[1], 3),
        'Macro Recall': round(baseline_metrics[2], 3),
        'Macro F1 Score': round(baseline_metrics[3], 3),
        'Models Tested': 0,  # No models tested for baseline
        'Best Fit Time (min)': 0  # No fit time for baseline
    })

    # For the rest of the models
    for model_info in models_list:
        # Create predictions
        y_pred = model_info['best_model'].predict(X_test)

        # Calculate metrics, using macro averages and accounting for possible zero division
        accuracy = accuracy_score(y_test, y_pred)
        macro_precision = precision_score(y_test, y_pred, average = 'macro', zero_division = 0)
        macro_recall = recall_score(y_test, y_pred, average = 'macro', zero_division = 0)
        macro_f1 = f1_score(y_test, y_pred, average = 'macro', zero_division = 0)

        # Convert fit time to minutes
        best_fit_time_minutes = model_info['best_model_fit_time'] / 60
        
        # Append record, rounding all float values to 3 decimals
        records.append({
            'Model Name': model_info['model_name'],
            'Accuracy': accuracy,
            'Macro Precision': macro_precision,
            'Macro Recall': macro_recall,
            'Macro F1 Score': macro_f1,
            'Models Tested': model_info['num_models_tested'],
            'Best Fit Time (min)': best_fit_time_minutes
        })

    # Create DataFrame to hold records
    report_df = pd.DataFrame(records)

    # Style DataFrame
    styled_report = (
        report_df.style
        .set_caption(
            "<b style='font-size:16px; color:white'>Overall Classification Report</b>"
        ) # Bold larger table title
        .format(precision = 4)  # Ensure 4 decimal places
        .highlight_max(subset = ['Accuracy', 'Macro Precision', 'Macro Recall', 'Macro F1 Score'], 
                       color = '#355C7D', axis = 0)  # Highlight best scores
        .set_table_styles([
            {
                'selector': 'th.col_heading', 'props':
                [('border-bottom', '2px solid white'), ('text-align', 'center'), ('color', 'white')]
            }, # White border underneath column headers, center and color white
            {
                'selector': 'caption', 'props': [('caption-side', 'top'), ('text-align', 'center')]
            }, # Center align table title
            {
                'selector': 'td:nth-child(1)',
                'props': [('border-right', '2px solid white')]
            },  # White border on the right of Model Name column
        ])
        .set_properties(**{
            'text-align': 'center',
            'color': 'white'
        }) # Center text and color white
        .hide(axis = 'index') # Hide index
    )
    
    return styled_report

In [16]:
# Helper function for evaluating training versus testing metrics
def training_vs_testing_metrics(models_list, X_train, y_train, X_test, y_test):
    records = []

    # Get metrics for random baseline on training and testing data
    baseline_train_metrics = generate_random_baseline(y_train)
    baseline_test_metrics = generate_random_baseline(y_test)

    # Append random baseline metrics
    records.append({
        'Model Name': 'Random Baseline',
        'Train Accuracy': baseline_train_metrics[0],
        'Test Accuracy': baseline_test_metrics[0],
        'Train Precision': baseline_train_metrics[1],
        'Test Precision': baseline_test_metrics[1],
        'Train Recall': baseline_train_metrics[2],
        'Test Recall': baseline_test_metrics[2],
        'Train F1 Score': baseline_train_metrics[3],
        'Test F1 Score': baseline_test_metrics[3]
    })

    # Get metrics for the rest of the models
    for model_info in models_list:
        # Predict on training and testing data
        y_train_pred = model_info['best_model'].predict(X_train)
        y_test_pred = model_info['best_model'].predict(X_test)

        # Calculate metrics for training and testing data
        train_accuracy = accuracy_score(y_train, y_train_pred)
        test_accuracy = accuracy_score(y_test, y_test_pred)

        train_precision = precision_score(y_train, y_train_pred, average = 'macro', zero_division = 0)
        test_precision = precision_score(y_test, y_test_pred, average = 'macro', zero_division = 0)

        train_recall = recall_score(y_train, y_train_pred, average = 'macro', zero_division = 0)
        test_recall = recall_score(y_test, y_test_pred, average = 'macro', zero_division = 0)

        train_f1 = f1_score(y_train, y_train_pred, average = 'macro', zero_division = 0)
        test_f1 = f1_score(y_test, y_test_pred, average = 'macro', zero_division = 0)

        # Append the record, rounding all values to 4 decimals
        records.append({
            'Model Name': model_info['model_name'],
            'Train Accuracy': train_accuracy,
            'Test Accuracy': test_accuracy,
            'Train Precision': train_precision,
            'Test Precision': test_precision,
            'Train Recall': train_recall,
            'Test Recall': test_recall,
            'Train F1 Score': train_f1,
            'Test F1 Score': test_f1
        })

    # Create DataFrame
    metrics_df = pd.DataFrame(records)

    # Reorder columns to ensure training and testing metrics together
    metrics_df = metrics_df[['Model Name',
                             'Train Accuracy', 'Test Accuracy',
                             'Train Precision', 'Test Precision',
                             'Train Recall', 'Test Recall',
                             'Train F1 Score', 'Test F1 Score']]

    # Style the DataFrame
    styled_df = (
        metrics_df.style
        .set_caption(
            "<b style='font-size:16px; color:white'>Classification Training vs. Test Scores</b>"
        )  # Bold larger table title
        .format(precision = 4) # Ensure 4 decimal places
        .set_table_styles([
            {
                'selector': 'th.col_heading', 'props':
                [('border-bottom', '2px solid white'), ('text-align', 'center'), ('color', 'white')]
            }, # White border underneath column headers, center and color white
            {
                'selector': 'caption', 'props': [('caption-side', 'top'), ('text-align', 'center')]
            }, # Center align table title
            {
                'selector': 'td:nth-child(1)',
                'props': [('border-right', '2px solid white')]
            },  # White border on the right of Model Name column
        ])
        .set_properties(**{
            'text-align': 'center',
            'color': 'white'
        })
        .hide(axis = "index")
    )
    
    return styled_df

In [17]:
# Helper function for generating per-class side-by-side comparisons for models as a Pandas Styled object
def generate_per_class_separate_tables(models_list, X_test, y_test, label_encoder):
    precision_records = []
    recall_records = []
    f1_records = []

    for model_info in models_list:
        # Extract model name
        model_name = model_info['model_name']

        # Create predictions
        y_pred = model_info['best_model'].predict(X_test)

        # Per-class metrics handling zero division
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_test, y_pred, average = None, zero_division = 0
        )

        # Classes based on label_encoder
        classes = label_encoder.inverse_transform(sorted(set(y_test)))
        
        for idx, cls in enumerate(classes):
            precision_records.append({'Class': cls, model_name: precision[idx]})
            recall_records.append({'Class': cls, model_name: recall[idx]})
            f1_records.append({'Class': cls, model_name: f1[idx]})

    # Convert to DataFrames
    precision_df = pd.DataFrame(precision_records).groupby('Class').first()
    recall_df = pd.DataFrame(recall_records).groupby('Class').first()
    f1_df = pd.DataFrame(f1_records).groupby('Class').first()

    # Styling function for DataFrames
    def style_table(df, metric_name):
        styled = (
            df.style
            .set_caption(
                f"<b style='font-size:16px; color:white'>{metric_name} per Class by Model</b>"
            )  # Bold larger table title
            .format(precision = 4)  # Keep 4 decimal places
            .highlight_max(axis = 1, color = '#355C7D')  # Highlight best scores
            .set_table_styles([
                {
                    'selector': 'th', 'props':
                    [('border-bottom', '2px solid white'), ('text-align', 'center'), ('color', 'white')]
                }, # White border under column level, center and color white
                {
                    'selector': 'th.col_heading', 'props':
                    [('border-bottom', '2px solid white'), ('text-align', 'center'), ('color', 'white')]
                }, # Second white border under column headers, center and color white
                {
                    'selector': 'th.row_heading', 'props':
                    [('border-right', '2px solid white'), ('text-align', 'center'), ('color', 'white')]
                }, # White border to the right of index which contains class names, center and color white
                {
                    'selector': 'caption', 'props': [('caption-side', 'top'), ('text-align', 'center')]
                } # Center align title
            ])
            .set_properties(**{
                'text-align': 'center',
                'color': 'white'
            }) # Center align text and color white
        )
        return styled

    # Style each DataFrame
    styled_precision = style_table(precision_df, "Precision")
    styled_recall = style_table(recall_df, "Recall")
    styled_f1 = style_table(f1_df, "F1 Score")

    return styled_precision, styled_recall, styled_f1

#### Evaluation

In [ ]:
# Fit using Google Colab - takes a long time to calculate
# plot_learning_curves(models, X_train, y_train, save_path = 'learning_curves.pkl')

# Loading saved figure from pickle

Viewing the overall classification scores, the Decision Tree and the XGBoost both performed very similarly. Although the Decision Tree had slightly better Macro F1, Precision and Accuracy compared to the XGBoost, the stats between the two models are nearly identical. Going strictly off of the main metric we have chosen for scoring (Macro F1), the Decision Tree is the winner, but the XGBoost model was faster to train and thus is likely more scalable for practically identical performance. Either of these two models seem appropriate.

The Random Forest performed the worst overall with a particularly low Macro Recall score. It also was the worst performing model in terms of fit time, meaning it is not likely to be scalable. The Logistic Regression performed slightly worse than the Decision Tree and XGBoost, but it had a much worse fit time than those models. For both of those reasons, Logistic Regression is not preferred.

All of the models significantly outperformed a baseline classifier that simply predicted randomly according to class probabilities.

In [18]:
generate_classification_report_table(models, X_test, y_test)

Model Name,Accuracy,Macro Precision,Macro Recall,Macro F1 Score,Models Tested,Best Fit Time (min)
Random Baseline,0.2730,0.1000,0.1000,0.1000,0,0.0000
SGDClassifier,0.8958,0.8501,0.7921,0.8082,27,6.4593
DecisionTreeClassifier,0.9035,0.8638,0.8051,0.8248,24,3.2537
RandomForestClassifier,0.8640,0.8695,0.6945,0.7431,8,11.2243
XGBClassifier,0.9023,0.8622,0.8072,0.8233,64,2.7853


Next, it is worthwhile to check for overfitting or underfitting in the models. There appears to be effectively no evidence of overfitting in the models, as the training and testing scores are nearly identical across metrics. Given the strength of the features that were crafted during the data preparation process, it would be hard to say that any of the models are underfitting except for perhaps the Random Forest.

Perhaps greater feature selection or engineering would produce better results, or maybe even more complex hyperparameter tuning, but generally speaking, macro scores in the 80-85%+ range would indicate that the models are performing very well in the face of large class imbalances (NYPD makes up 46% of the data, HPD makes up 21% of the data, and the next highest class representation is the DSNY at 9% of the data). Considering the extremely poor performance of a baseline model that randomly predicts according to class probabilities, it seems safe to say that the models are not underfitting.

In [19]:
training_vs_testing_metrics(models, X_train, y_train, X_test, y_test)

Model Name,Train Accuracy,Test Accuracy,Train Precision,Test Precision,Train Recall,Test Recall,Train F1 Score,Test F1 Score
Random Baseline,0.2736,0.2735,0.1000,0.1001,0.1000,0.1001,0.1000,0.1001
SGDClassifier,0.8955,0.8958,0.8496,0.8501,0.7912,0.7921,0.8073,0.8082
DecisionTreeClassifier,0.9049,0.9035,0.8677,0.8638,0.8082,0.8051,0.8280,0.8248
RandomForestClassifier,0.8640,0.8640,0.8704,0.8695,0.6944,0.6945,0.7431,0.7431
XGBClassifier,0.9024,0.9023,0.8631,0.8622,0.8077,0.8072,0.8238,0.8233


The next step is to identify class-by-class performance to see where each model struggled.

In [20]:
precisions, recalls, f1s = generate_per_class_separate_tables(models, X_test, y_test, label_encoder)

The Random Forest had the best overall performance with regards to precision. Notably, the Random Forest performed particularly well on the lower frequency classes of DHS, DOHMH, and Other. However, the Random Forest also struggled with precision on the most common classes of NYPD and HPD, at least in comparison to the other classifiers, which means the Random Forest was overconfident about the high frequency classes. The XGBoost model has the highest precision on the NYPD, meaning it is the most likely to not misdirect calls to the NYPD and waste valuable police resources.

All of the other classifiers performed relatively similarly across classes. The DOB and DPR were the hardest classes to predict precisely.

In [21]:
precisions

,SGDClassifier,DecisionTreeClassifier,RandomForestClassifier,XGBClassifier
Class,,,,
DEP,0.9517,0.9503,0.9405,0.9570
DHS,0.7541,0.7804,0.8048,0.7678
DOB,0.7501,0.7741,0.7533,0.7631
DOHMH,0.8241,0.8399,0.8818,0.8398
DOT,0.9698,0.9394,0.9669,0.9604
DPR,0.7302,0.7586,0.7356,0.7444
DSNY,0.7836,0.8332,0.9244,0.8101
HPD,0.9804,0.9783,0.9599,0.9774
NYPD,0.9016,0.9028,0.8254,0.9059


The Random Forest clearly favors the majority classes, achieving nearly perfect recall on both NYPD and HPD. The XGBoost model has the best overall recall and performed quite a bit better on the DHS category than the other models. Interestingly, the Logistic Regression outperforms the Decision Tree in a number of classes (DOB, DPR, and DSNY).

All of the classes, besides the Random Forest, performed fairly similarly across classes. The hardest classes to attain high recall on were DOT, Other, and DHS. Fortunately, all of the models had fairly high recall on the majority classes of the HPD and NYPD, as it is important for the most common agencies to actually receive calls that should be directed to them.

In [22]:
recalls

,SGDClassifier,DecisionTreeClassifier,RandomForestClassifier,XGBClassifier
Class,,,,
DEP,0.8559,0.8689,0.8524,0.8620
DHS,0.6652,0.6863,0.4557,0.7002
DOB,0.9977,0.9578,0.9803,0.9811
DOHMH,0.7163,0.7818,0.4275,0.7735
DOT,0.5538,0.5829,0.5571,0.5677
DPR,0.9140,0.9020,0.8252,0.9168
DSNY,0.7205,0.7070,0.3678,0.7249
HPD,0.9831,0.9931,0.9932,0.9931
NYPD,0.9652,0.9718,0.9909,0.9665


When looking at the F1 scores, it becomes immediately obvious that the Decision Tree and the XGBoost model are extremely close. The Random Forest has frankly terrible scores in comparison, and the Logistic Regression lags behind the Decision Tree and the XGBoost model. The XGBoost model has the slight advantage in F1 score for the rarest class (DHS), but the Decision Tree outperforms in far more classes. However, the difference between the two is so slight that they are functionally the same.

In [23]:
f1s

,SGDClassifier,DecisionTreeClassifier,RandomForestClassifier,XGBClassifier
Class,,,,
DEP,0.9013,0.9077,0.8943,0.9070
DHS,0.7069,0.7304,0.5819,0.7325
DOB,0.8563,0.8562,0.8519,0.8585
DOHMH,0.7664,0.8099,0.5758,0.8053
DOT,0.7050,0.7194,0.7069,0.7136
DPR,0.8118,0.8241,0.7778,0.8216
DSNY,0.7507,0.7649,0.5262,0.7652
HPD,0.9817,0.9856,0.9763,0.9852
NYPD,0.9323,0.9360,0.9006,0.9352


Overall, either the Decision Tree or the XGBoost model would be suitable choices for directing 311 calls to various NY agencies. Their performance is very similar across the board. The Decision Tree is useful in that it is more interpretable. The decision tree itself can be generated and shown to analyze all of the steps that make up its decision making process. The XGBoost model has the benefit of being able to utilize GPUs while training, making it more likely to scale as the size of the data grows, since it can leverage more powerful hardware to gain performance over the normal sci-kit learn models.

As the full size of the 311 dataset is very large and only continues to grow over time, the computational efficiency of the XGBoost model is likely more important than the interpretability of the Decision Tree model.